# Diffusion equations

In [ ]:
import numpy as np

from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Linear diffusion equation (heat equation, Dynamics and Relativity Sheet 1 Question 8)

The diffusion equation
$$ y_t = D y_{xx} $$
on the infinite domain $x \in \mathbb{R}$, with $y \rightarrow 0$ as $x \rightarrow \pm\infty$, has the similarity solution
$$
y_s(x, t) = \frac{Q}{\sqrt{4\pi D t}} \exp\left(\frac{-x^2}{4Dt}\right), \qquad
Q = \int_{-\infty}^{\infty} y(x, 0) \, \mathrm{d}t
$$
Given any initial condition $y(x, 0)$ that is localised, it can be shown that, for $t\gg1$, the solution $y(x,t)$ approaches the similarity solution.

(The precise definition of this is subtle, and the result can be derived using Fourier transforms and Laplace's method for estimating large integrals. This is covered in detail in _Methods_ and _Asymptotic Methods_.)

The similarity solution can be derived using dimensional analysis. For $t\gg1$ it is assumed that all the details about the initial conditions are 'forgotten' due to the smoothing nature of diffusion; therefore, the resulting solution at a point $x$ may depend only on $x$, $t$, $D$ and the total quantity $Q = \int_{-\infty}^\infty y \,\mathrm{d}t$, which is conserved. The only way in which $x$, $t$ and $D$ may be combined into a dimensionless quantity is through $s = x/\sqrt{Dt}$. 

In this demo, we have chosen units in which $D = 1$.

In [ ]:
dx = 0.01
xs = np.arange(-10, 10, dx)


def y0(x):
    return np.where(abs(x) < 1, 1 + 0.3*x + 0.2 * np.sin(6*x), 0)

y0s = y0(xs)

def dydt(t, ys):
    d2ydx2 = (np.roll(ys, -1) - 2*ys + np.roll(ys, 1)) / dx**2
    return d2ydx2


sol = solve_ivp(dydt, (0, 6), y0s)
assert sol.success, sol.status

ysol = interp1d(sol.t, sol.y)

@interact(t=widgets.FloatSlider(min=sol.t[0], max=sol.t[-1], step=0.01, continuous_update=False))
def diffusion_spread(t):
    q = np.sum(y0s) * dx
    if t > 0:
        y_sim = q * (4 * np.pi * t)**(-0.5) * np.exp(-xs**2 / (4*t))
    else:
        y_sim = np.where(np.abs(xs) < dx, q / dx, 0)
        
    fig, axs = plt.subplots()

    plt.plot(xs, ysol(t), 'k-')
    plt.plot(xs, y_sim, 'r--')
    axs.set_ylim([0, np.max(sol.y) + 0.1])
    axs.legend(('Actual solution', 'Similarity solution'))
    plt.show()

One problem with the linear diffusion equation as a model is that 'information travels infinitely fast' - that is, we can have $y(x, t) \neq 0$ for arbitrarily large $x$ and arbitrary small $t$ despite $y(x, 0)$ being localised. This behaviour is not exhibited by nonlinear diffusion equations...

## Nonlinear diffusion (Mathematical Biology Sheet 3 Question 6)

$$ C_t = k (C^p C_x)_x $$

We see that when the diffusion coefficient is not constant but depends on $C$ then 'information does not travel infinitely fast'. Instead, a front forms.

In [ ]:
dx = 0.05
xs = np.arange(-10, 10, dx)

y0s = np.where(abs(xs) < 1, 1 + np.sin(xs), 0)

def dydt(t, ys):
    yx = np.concatenate([[0], (ys[2:] - ys[:-2])/(2*dx), [0]])
    yxx = np.concatenate([[0], (ys[2:] - 2*ys[1:-1] + ys[:-2])/dx**2, [0]])
    return yx**2 + ys * yxx

sol = solve_ivp(dydt, (0, 10), y0s, method='Radau')
assert sol.success, sol.message

ysol = interp1d(sol.t, sol.y)

@interact(t=widgets.FloatSlider(min=sol.t[0], max=sol.t[-1], continuous_update=False))
def diffusion_spread(t):
    fig, axs = plt.subplots()

    plt.plot(xs, ysol(t))
    axs.set_ylim([0, np.max(sol.y) + 0.1])
    plt.show()